In [1]:
import pandas as pd
import numpy as np
import os
import warnings

warnings.filterwarnings("ignore")
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import learning_curve
from sklearn.model_selection import validation_curve
from sklearn.metrics import classification_report
from catboost import CatBoostClassifier, Pool

In [2]:
data = pd.read_csv(
    "../data/external/public_train_features.csv", index_col="row_id"
)
data.drop(
    columns=[
        "timestamp",
        "timestamp_lag_1",
        "timestamp_lag_2",
        "timestamp_lag_3",
        "timestamp_lag_4",
    ],
    inplace=True,
)
data.fillna(value=0, inplace=True)

In [3]:
lst_to_norm = [
    "diff_time_gate_lag_1",
    "diff_time_gate_lag_2",
    "diff_time_gate_lag_3",
    "diff_time_gate_lag_4",
    "minute",
    "second",
    "day",
    "time_to_sec",
    "dayweek",
    "minute_lag_1",
    "minute_lag_2",
    "minute_lag_3",
    "minute_lag_4",
    "second_lag_1",
    "second_lag_2",
    "second_lag_3",
    "second_lag_4",
    "day_lag_1",
    "day_lag_2",
    "day_lag_3",
    "day_lag_4",
    "time_to_sec_lag_1",
    "time_to_sec_lag_2",
    "time_to_sec_lag_3",
    "time_to_sec_lag_4",
    "dayweek_lag_1",
    "dayweek_lag_2",
    "dayweek_lag_3",
    "dayweek_lag_4",
    "diff_time_to_sec_2",
    "diff_time_to_sec_3",
    "diff_time_to_sec_4",
]

In [4]:
data[lst_to_norm].describe()

,diff_time_gate_lag_1,diff_time_gate_lag_2,diff_time_gate_lag_3,diff_time_gate_lag_4,minute,second,day,time_to_sec,dayweek,minute_lag_1,...,time_to_sec_lag_2,time_to_sec_lag_3,time_to_sec_lag_4,dayweek_lag_1,dayweek_lag_2,dayweek_lag_3,dayweek_lag_4,diff_time_to_sec_2,diff_time_to_sec_3,diff_time_to_sec_4
count,4.464300e+04,4.464300e+04,4.464300e+04,4.464300e+04,44643.000000,44643.000000,44643.000000,44643.000000,44643.000000,44643.000000,...,44643.000000,44643.000000,44643.000000,44643.000000,44643.000000,44643.000000,44643.000000,44643.000000,44643.000000,44643.000000
mean,5.649550e+03,1.114624e+04,1.605617e+04,2.088882e+04,29.182470,29.468136,15.715431,52937.300943,2.157561,29.181484,...,52934.117958,52932.526645,52930.935354,2.157471,2.157382,2.157292,2.157203,1.706203,2.558453,3.410411
std,6.492905e+04,9.272200e+04,1.076808e+05,1.207986e+05,17.269915,17.319715,9.086933,13117.605029,1.500084,17.270325,...,13121.829271,13123.940840,13126.051908,1.500093,1.500103,1.500112,1.500121,3910.069039,4784.678004,5525.638387
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,1.000000,3172.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-81630.000000,-81624.000000,-81623.000000
25%,2.000000e+00,5.010000e+02,9.230000e+02,1.531000e+03,14.000000,15.000000,7.000000,41104.000000,1.000000,14.000000,...,41102.000000,41102.000000,41102.000000,1.000000,1.000000,1.000000,1.000000,19.000000,37.000000,76.000000
50%,2.890000e+02,1.353000e+03,2.248000e+03,3.271000e+03,29.000000,29.000000,16.000000,52045.000000,2.000000,29.000000,...,52043.000000,52041.000000,52041.000000,2.000000,2.000000,2.000000,2.000000,52.000000,155.000000,286.000000
75%,1.404000e+03,3.356000e+03,5.198000e+03,7.286500e+03,44.000000,44.000000,23.000000,64807.000000,3.000000,44.000000,...,64806.000000,64806.000000,64806.000000,3.000000,3.000000,3.000000,3.000000,298.000000,531.000000,741.000000
max,7.077913e+06,7.180401e+06,8.545990e+06,1.236009e+07,59.000000,59.000000,31.000000,85722.000000,6.000000,59.000000,...,85722.000000,85722.000000,85722.000000,6.000000,6.000000,6.000000,6.000000,52805.000000,52807.000000,55152.000000


In [5]:
target = pd.read_csv("../data/interim/target.csv", index_col="row_id")

In [6]:
data_train = data.join(target, on=None, how="inner").iloc[4:]

In [7]:
data_test = data.loc[37518:]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    data_train.drop(columns=["user_id"]),
    data_train["user_id"],
    random_state=15,
    test_size=0.25,
)

In [9]:
# pipe = make_pipeline(
#     StandardScaler(), LogisticRegression(n_jobs=-3, solver="saga", max_iter=200, multi_class="ovr", C=0.01)
# )
# pipe.fit(X_train, y_train)

# print(pipe.score(X_test, y_test) * 100)
# print(classification_report(y_test, pipe.predict(X_test)))
# # 17.304616696876

In [14]:
clf = CatBoostClassifier(
    thread_count=4,
    iterations=2000,
    early_stopping_rounds=50,
    eval_metric="TotalF1",
    use_best_model=True,
    learning_rate=0.01,
)
clf.fit(X_train, y_train, verbose=False, eval_set=(X_test, y_test))

In [11]:
stop

NameError: name 'stop' is not defined

In [15]:
data_test["target"] = clf.predict(data_test)
data_test["target"]

row_id
37518    18
37519    18
37520     3
37521     3
37522    15
         ..
44638    37
44639     6
44640    19
44641    54
44642    54
Name: target, Length: 7125, dtype: int64

In [16]:
data_out = data_test["target"]
data_out.to_csv("../data/processed/sample_submission_2.sv")